In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
import os

In [2]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0
)
# llm = ChatOllama(
#     model="qwen2:latest",
#     temperature=0,
#     base_url="http://localhost:11434",
# )

In [38]:
from langchain.tools.retriever import create_retriever_tool

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_NEWS_INDEX = "MY_NEWS_INDEX"
vectorstore1 = FAISS.load_local(MY_NEWS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="saved_news_search",
    description="""
아래와 같은 정보를 검색할 때에는 이 도구를 사용해야 한다
- 엔비디아의 스타트업 인수 관련 내용
- 퍼플렉시티 관련 내용 (회사가치, 투자 등)
- 라마3 관련 내용
""",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="""
다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다
- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용
- 생성 AI 규제 연구 관련 내용
- 생성 AI 연구 관련 내용
"""
)

tools = [retriever_tool1, retriever_tool2]
tools

[Tool(name='saved_news_search', description='\n아래와 같은 정보를 검색할 때에는 이 도구를 사용해야 한다\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10af4fec0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x307f29f90>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10b1200e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x307f29f90>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description='

In [26]:
json_prompt = hub.pull("teddynote/react-chat-json-korean")
json_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'react-chat-json-korean', 'lc_hub_commit_hash': '246b7852986ad9370b8d41310461e4e52a947ea1b67bf034722f4447a4caa064'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate hum

In [44]:
from langchain.agents import create_json_chat_agent
from langchain.agents import AgentExecutor

agent = create_json_chat_agent(llm, tools, json_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

In [46]:
response = agent_executor.invoke(
    {"input": "퍼플렉시티가 투자받은 금액?"}
    # {"input": "생성형 AI 도입에 따른 규제 연구 책임자는?"}
) 
response["output"]



> Entering new AgentExecutor chain...
```
{
    "action": "saved_news_search",
    "action_input": "퍼플렉시티의 투자금액"
}
```인공지능(AI)을 활용해 구글과 경쟁할 검색 엔진을 구축하는 스타트업 퍼플렉시티가 최근 약 6300만달러(약 860억원)의 투자를 유치하며 회사 가치를 10억달러(약 1조3760억원) 이상으로 평가 받았다. 3개월 만에 이 회사의 기업 가치는 두배로 상승했다.
테크크런치에 따르면 23일(현지시각) 발표된 이번 투자에는 다니엘 그로스, 스탠리 드뤼컨밀러, Y 콤비네이터의 최고 경영자 게리 탄, 그리고 피그마 CEO 딜런 필드 등 거물들이 새로 참여했다. 아마존 창업자인 제프 베이조스와 엔비디아 등 기존 투자자들도 이번 라운드에 이름을 올렸다.
이번 자금 조달이 마감됐지만, 퍼플렉시티는 현재 25억달러(약 34조원)에서 30억달러(약 41조원)의 기업가치로 최소 2억5000만달러(약 3400억원)의 훨씬 더 큰 규모의 라운드를 진행 중인 것으로 알려졌다.

새로운 투자 라운드가 성공적으로 마감된다면 지난 1월 5억4000달러(약 7400억원) 가치로 7400만달러(약 1000억원)를 모금한 이후 기업가치가 5~6배까지 상승하는 것이다. 
퍼플렉시티는 창업한 지 2년도 채 안 됐지만, 검색 결과를 요약하는 AI 챗봇으로 큰 인기를 얻고 있다. 구글의 제미나이, 미스트랄 7B, 앤트로픽의 클로드, 오픈AI의 GPT-4, 메타의 라마 3 등 여러 주요 LLM을 활용하여 최상의 검색 결과를 제공한다.
답변에 대한 인용을 나열하고 사용자가 최상의 응답을 얻도록 돕는 서비스를 제공, 구글 검색의 대안으로 꼽힌다. 정확성에 중점을 두고 있으며, 점점 더 혼잡해지는 AI 시장에서 신속하게 자신의 챗봇을 차별화하고 있다. 
또 무료 및 유료 버전의 서비스를 통해 연간 2000만달러(약 275억원)의 수익을 올리고 있다. 다른 많은 AI 기업들과 마찬가지로 B2B로 수익

'퍼플렉시티가 약 6300만달러(약 860억원)의 투자를 유치하며 회사 가치를 10억달러(약 1조3760억원) 이상으로 평가 받았다.'